In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle as pkl
from collections import defaultdict,deque,Counter,OrderedDict
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
import os
import time
import copy
import random

from models import LM_latent, LM_latent_type_rep
from vocab import Vocabulary

from evaluate import *

In [7]:
torch.manual_seed(9090)
cudnn.benchmark = True
    
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    device = 'cuda'
else:
    device = 'cpu'

In [19]:
test_pickle_file = '/misc/vlgscratch4/BrunaGroup/rj1408/nlu/ptb_wsj_pos/test.p'
model_left_to_right = '/misc/vlgscratch4/BrunaGroup/rj1408/nlu/ptb_wsj_pos/models/base_lstm_defpar/a/'
model_right_to_left = '/misc/vlgscratch4/BrunaGroup/rj1408/nlu/ptb_wsj_pos/models/base_lstm_defpar_reverse/a/'

with open(test_pickle_file,"rb") as a:
    testdict = pkl.load(a)

#Load forward model
forward_model_params = torch.load(os.path.join(model_left_to_right, 'net_epoch_0.pth'), map_location=device)
forward_model_weights = torch.load(os.path.join(model_left_to_right, 'net_best_weights.pth'), map_location=device)
tag2id = forward_model_params["hyperparams"]["tagtoid"]
id2tag = defaultdict(str)
for tag in tag2id:
    id2tag[tag2id[tag]] = tag
vocab = forward_model_params["hyperparams"]["vocab"]
hidden_size = forward_model_params["hyperparams"]["hidden_size"]
tok_emb_size = forward_model_params["hyperparams"]["token_embedding"]
tag_emb_size = forward_model_params["hyperparams"]["tag_emb_size"]
lstm_layers = forward_model_params["hyperparams"]["lstmLayers"]
dropout_p =  0.1

tag_wise_vocabsize = dict([(tag2id[tup[0]], tup[1][2]) for tup in vocab.tag_specific_vocab.items()])
forward_model = LM_latent(vocab.vocab_size, tag_wise_vocabsize, hidden_size, tok_emb_size, tag_emb_size, lstm_layers, dropout_p)
forward_model.load_state_dict(forward_model_weights)
forward_model = forward_model.to(device)

#Load backward model
backward_model_params = torch.load(os.path.join(model_right_to_left, 'net_best_weights.pth'), map_location=device)
tag2id = backward_model_params["hyperparams"]["tagtoid"]
id2tag = defaultdict(str)
for tag in tag2id:
    id2tag[tag2id[tag]] = tag
vocab = backward_model_params["hyperparams"]["vocab"]
hidden_size = backward_model_params["hyperparams"]["hidden_size"]
tok_emb_size = backward_model_params["hyperparams"]["token_embedding"]
tag_emb_size = backward_model_params["hyperparams"]["tag_emb_size"]
lstm_layers = backward_model_params["hyperparams"]["lstmLayers"]
dropout_p =  0.1

tag_wise_vocabsize = dict([(tag2id[tup[0]], tup[1][2]) for tup in vocab.tag_specific_vocab.items()])
backward_model = LM_latent(vocab.vocab_size, tag_wise_vocabsize, hidden_size, tok_emb_size, tag_emb_size, lstm_layers, dropout_p)
backward_model.load_state_dict(backward_model_params["model_state_dict"])
backward_model = backward_model.to(device)

In [20]:
UNKNOWN_TAG = tag2id['UNKNOWN']
PAD_TAG_ID = -51